In [13]:
import requests
import time
from bs4 import BeautifulSoup
# import forecastio
import json
from pandas.io.json import json_normalize

### request
- 1) json -- if there is api
- 2) html -(parsing by using css selector for crawling)- bs4(string data)
- 3) selenium -- css selector (by using css selector and xpath)
- 4) scrapy - package for crawling, use css selector, xpath 
- number of each row is about priority in using method.

##### 네이버 주식 데이터 가져오기
- api 사용 : json 파싱을 합니다.
- 네이버 주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만듭니다.
- 웹서버보다 모바일 페이지가 크롤링이 더 쉬운 경우가 있으므로 염두에 둡니다.

url 분석

In [ ]:
url = "http://m.stock.naver.com/api/json/sise/\
siseListJson.nhn?menu=market_sum&sosok={0}&pageSize={0}&page={}"

#### 분석 결과
- sosok = 보여지는 주식 카테고리(코스피,코스닥...)
- pageSize = 한번에 보여지는 주식 데이터의 갯수
- page = 더보기를 통해 보여지는 주식이 달라지지는데 그 순서를 의미합니다.

- result: {
    - totCnt: 1274,
    - ms: "OPEN",
    - itemList: [
- result 보다 낮은 level로 factor들이 들어있는 data format을 가지고 있습니다.

make_url 함수 만들기 -- 인자에 따라서 보여지는 url의 json data가 바뀝니다.

In [ ]:
http://m.stock.naver.com/api/json/sise/siseListJson.nhn?m\
    enu=market_sum&sosok=1&pageSize=5&page=5

In [1]:
def make_url(pageSize=10,page=1):
    return 'http://m.stock.naver.com/api/json/sise\
/siseListJson.nhn?menu=market_sum&sosok=0&pageSize='+str(pageSize)+'&page='+str(page)
#default argument를 정해주고 후에 필요시, 변경해주도록 합니다.

In [16]:
make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

받은 json data를 parsing해주고 DataFrame 으로 만들어주는 함수 get_data 만들기

In [30]:
def get_data_normalize(url):
    response = requests.get(url)
    jo = json.loads(response.text)
    company = jo['result']['itemList']
    result = json_normalize(company)
    return result

In [22]:
def get_data(url):
    response = requests.get(url) # url 을 요청합니다.
    json_info = response.json() # json data format으로 parsing
    companys = json_info['result']['itemList'] #우리가 필요한 데이터는 itemList 배열입니다.
    df = pd.DataFrame(columns = ['종목','시세','전열비','등락율','시가총액','거래량'])
    #df이라는 dataframe 변수를 생성해주고 column을 할당해줍니다. 여기서 columns의 갯수는
    #데이터의 갯수와 같게 설정해주어야 합니다.
    for company in companys:
        df.loc[len(df)] = { #위에서 부터 하나씩 아래로 데이터들을 for 반복문을 통해 채워나갑니다.
            '종목':company['nm'],
            '시세':company['nv'],
            '전열비':company['cv'],
            '등락율':company['cr'],
            '시가총액':company['mks'],
            '거래량':company['aq'],
        }#dictionary datatype으로 dataframe의 value들을 채워넣어가 줍니다.
    return df

In [24]:
url = make_url(1406,1) #1406(kospi corp vol), 1(first page)
df = get_data(url)
print(len(df))
df.head()

1406


,종목,시세,전열비,등락율,시가총액,거래량
0,삼성전자,47650,-550,-1.14,3058808,16802342
1,SK하이닉스,87000,700,0.81,633362,2931145
2,셀트리온,298500,18500,6.61,374307,1820384
3,삼성전자우,37800,-950,-2.45,341572,2402033
4,POSCO,365000,-4500,-1.22,318232,232916
